# Pharmacy counting challenge of the Insight Data Engineering program
### Solution by Madlen Wilmes

## Problem description

Efficiently process a large data file that provides the following information:

id: prescriber id<br>
*Prescriber_last*: Last name of physician that prescribed the medication<br>
*prescriber_first_name*: First name of physician that prescribed the medication<br>
*drug_name*: Name of the prescribed drug<br>
*drug_cost*: Cost of the prescription<br>

Special request: Use basic Python data structures, no libraries beyond file in-/output, implement proper unit testing

**Example input file:**

id,prescriber_last_name,prescriber_first_name,drug_name,drug_cost<br>
1000000001,Smith,James,AMBIEN,100<br>
1000000002,Garcia,Maria,AMBIEN,200<br>
1000000003,Johnson,James,CHLORPROMAZINE,1000<br>
1000000004,Rodriguez,Maria,CHLORPROMAZINE,2000<br>
1000000005,Smith,David,BENZTROPINE MESYLATE,1500

Output the results as comma-separated file with the following information:

*drug_name*: the exact drug name as shown in the input dataset<br>
*num_prescriber*: the number of unique prescribers who prescribed the drug. For the purposes of this challenge, a prescriber is considered the same person if two lines share the same prescriber first and last names<br>
*total_cost*: total cost of the drug across all prescribers

**Example output:**

drug_name,num_prescriber,total_cost<br>
CHLORPROMAZINE,2,3000<br>
BENZTROPINE MESYLATE,1,1500<br>
AMBIEN,2,300


## Data exploration using bash

In [8]:
! wc -l ../input/de_cc_data.txt
! head ../input/de_cc_data.txt

 24525860 ../input/de_cc_data.txt
id,prescriber_last_name,prescriber_first_name,drug_name,drug_cost
1952310666,A'BODJEDI,ENENGE,ALPRAZOLAM,1964.49
1952310666,A'BODJEDI,ENENGE,AMANTADINE,1129.94
1952310666,A'BODJEDI,ENENGE,AMBIEN,4792.29
1952310666,A'BODJEDI,ENENGE,AMBIEN CR,4792.29
1952310666,A'BODJEDI,ENENGE,AMITRIPTYLINE HCL,692.78
1952310666,A'BODJEDI,ENENGE,ARIPIPRAZOLE,8374.44
1952310666,A'BODJEDI,ENENGE,BENZTROPINE MESYLATE,1868.72
1952310666,A'BODJEDI,ENENGE,BUPROPION XL,3111.04
1952310666,A'BODJEDI,ENENGE,BUSPIRONE HCL,837.7


In [15]:
# check number of unique medications in first 100.000 lines
! tail -n +2  ../input/de_cc_data.txt | head -n 100000 | cut -d "," -f 4 | sort | uniq | wc -l

tail: stdout: Broken pipe
    1400


In [19]:
# check number of unique prescribers in first 100.000
! tail -n +2  ../input/de_cc_data.txt | head -n 100000 | cut -d "," -f 2,3 | sort -k3,2 | head
! tail -n +2  ../input/de_cc_data.txt | head -n 100000 | cut -d "," -f 3,2 | sort -k 4,3 | uniq | wc -l

tail: stdout: Broken pipe
"ABRAHAM, M.D."
"ABRAHAM, M.D."
"ABRAHAM, M.D."
"ABRAHAM, M.D."
"ABRAHAM, M.D."
"ABRAHAM, M.D."
"ABRAHAM, M.D."
A'BODJEDI,ENENGE
A'BODJEDI,ENENGE
A'BODJEDI,ENENGE
sort: write failed: standard output: Broken pipe
sort: write error
tail: stdout: Broken pipe
    3230


In [21]:
# some entries contain commas, enclosed in quotes
! grep '"' ../input/de_cc_data.txt | cut -d "," -f 2-4 | sort | uniq | head

"ABRAHAM, M.D.",CINI
"ADAIR,",ROBERT
"ADAMS, II",WILLIAM
"ADAMS, JR.",WILLIE
"ALCARAZ, JR",RENATO
"ALIZADEH,MD INC.",AMIR
"ALLEN., III",WILLIAM
"AMULURU, MD",LAVANYA
"ANDERSON III,",WILLIAM
"ASHLEY,",BRYANT


In [103]:
# how many prescribers according to id?
! cut -d ',' -f1 ../input/de_cc_data.txt | sort -nr | head -1

1992999874
sort: write failed: standard output: Broken pipe
sort: write error


In [72]:
# generate a truncated data file with 100.000 records
! head -n 1 ../input/de_cc_data.txt > ../input/data_trunc.txt
! tail -n +70560 ../input/de_cc_data.txt | head -n 10000  >> ../input/data_trunc.txt

tail: stdout: Broken pipe


In [1]:
import csv

In [21]:
# read in data file
pharm = {}
with open("../input/itcont.txt", "r") as f:
#with open("../input/data_trunc.txt", "r") as f:
    my_csv = csv.DictReader(f)
    for line in my_csv:
        drug = line["drug_name"]
        prescriber = tuple([line["prescriber_first_name"], line["prescriber_last_name"]])
        cost = float(line["drug_cost"])
        pharm[drug] = pharm.get(drug, [set(),float()])
        pharm[drug][0].add(prescriber)
        pharm[drug][1] = pharm[drug][1] + cost

In [82]:
len(pharm)

2749

In [28]:
list(pharm.items())[0]

('ALPRAZOLAM',
 [{('CHUN', 'HWANG'),
   ('PREM', 'SAHASRANAM'),
   ('ROBERT', 'GARFIELD'),
   ('THERESA', 'ARTHUR'),
   ('AMIT', 'PRADHAN'),
   ('JAMES', 'BEANE'),
   ('BRUCE', 'RICHMAN'),
   ('TRINA', 'KAISER'),
   ('TIMOTHY', 'VOLM'),
   ('JOHN', 'WHEAT'),
   ('ALISON', 'GOMEZ'),
   ('SUKHPAL', 'GILL'),
   ('JOSEPH', 'LIOTTA'),
   ('TAMI', 'MOORE'),
   ('JOHN', 'BAGA'),
   ('JUDE GERARD', 'VERZOSA'),
   ('RALPH', 'MAY'),
   ('RYAN', 'TYLER'),
   ('STEPHEN', 'SHY'),
   ('SHAHEEN', 'PIRANI'),
   ('PHYLIS', 'MUTHEE'),
   ('RUDOLPH', 'PUANA'),
   ('JONATHAN', 'HAFFNER'),
   ('LAURA', 'VARLARO'),
   ('ISMET', 'MIRZA'),
   ('RAYMOND', 'HANSEN'),
   ('MAIJA', 'SANNA'),
   ('JONATHAN', 'LANGBERG'),
   ('BERNICE', 'WONG'),
   ('DENISE', 'HOM'),
   ('JOHN', 'STANKO'),
   ('LINDA', 'BURRELL'),
   ('PAUL', 'KELLEY'),
   ('JILL', 'BRESSLER'),
   ('JEROME', 'WALLER'),
   ('SUJATHA', 'GOVINDAIAH'),
   ('IKRAM', 'HUSSAIN'),
   ('YOON', 'MIN'),
   ('SHARON', 'GRAFF'),
   ('LASZLO', 'PAPP'),
   ('ANSH

In [22]:
sort_pharm = sorted(pharm.items(), key = lambda x: (-x[1][1],x[0]))

In [24]:
sort_pharm[0]

('HARVONI',
 [{('ANOOP', 'GOYAL'),
   ('JULIE', 'CHANG'),
   ('SHARON', 'LEE'),
   ('STEVANY', 'PETERS'),
   ('NICOLE', 'SCHOCK'),
   ('BRUCE', 'TUNG'),
   ('KENNETH', 'BERMAN'),
   ('SHAD', 'JOHNSON'),
   ('IBRAHIM', 'HANOUNEH'),
   ('CAROLL', 'KOSCHESKI'),
   ('PAUL', 'BIERMAN'),
   ('STEPHEN', 'JOHNSON'),
   ('MANJUSHREE', 'GAUTAM'),
   ('CARRIE', 'TURNER'),
   ('PRATIMA', 'SHARMA'),
   ('ANNA', 'RUTHERFORD'),
   ('RICHARD', 'STERLING'),
   ('ERIN', 'NOMLAND'),
   ('MOHAMMED EYAD', 'YASEEN ALSABBAGH'),
   ('IRTEZA', 'INAYAT'),
   ('OLUSEGUN', 'ADEONIGBAGBE'),
   ('DEVON', 'JONES'),
   ('MARNI', 'MARTINEZ'),
   ('ALBERT', 'FERNANDEZ BRAVO'),
   ('KANAN', 'SHARMA'),
   ('SHARAD', 'BELLAPRAVALU'),
   ('ABDULLAH', 'MUBARAK'),
   ('WINDY', 'ADAMS'),
   ('LESLIE', 'DIAZ'),
   ('LINDA', 'DITEODORO'),
   ('JAGRAJ', 'NIJJAR'),
   ('SHOBHA', 'JOSHI'),
   ('ROBERTA', 'ROMERO'),
   ('AZADE', 'YEDIDAG'),
   ('ROBERT', 'KODSI'),
   ('LAURIE', 'EINWALD'),
   ('LORNA', 'DOVE'),
   ('HOWARD', 'ESTRI

In [23]:
with open("../output/top_cost_drug.txt","w") as outfile:
    outfile.write("drug_name,num_prescriber,total_cost")
    for drug in sort_pharm:
        outfile.write("\n{},{},{:.0f}".format(drug[0], len(drug[1][0]), drug[1][1],2))

In [4]:
# how many unique prescriber by (first, last)
uniq_prescribers = set()
for drug in pharm.values():
    uniq_prescribers.update(drug[0])

In [17]:
len(uniq_prescribers)

733767

In [18]:
# instead of using unruly list within the pharm dictionary, also a named tuple would have made sense
import collections
DrugData = collections.namedtuple('DrugData', ['prescribers', 'cost'])

In [19]:
pharm1 = {}
pharm1['VALIUM'] = DrugData(set(), 0)
pharm1['VALIUM'].cost

0